In [1]:
# inserting in location some values

import theano 
import theano.tensor as T
import numpy
T_location = T.imatrix("location")
T_values = T.vector("values")
output_model = T.matrix("output_model")

def step(a_location, a_values, output_model):
    return T.set_subtensor(output_model[a_location[0], a_location[1]], a_values)

result, updates = theano.scan(fn=step,
                              outputs_info=T.zeros((5, 5)),
                              sequences=[T_location, T_values])

assign_values_at_positions = theano.function(inputs=[T_location, T_values], outputs=result[-1])

# test
test_locations = numpy.asarray([[1, 1], [2, 3]], dtype=numpy.int32)
test_values = numpy.asarray([42, 50], dtype=numpy.float32)
test_output_model = numpy.zeros((5, 5), dtype=numpy.float32)
print(assign_values_at_positions(test_locations, test_values))




[[  0.   0.   0.   0.   0.]
 [  0.  42.   0.   0.   0.]
 [  0.   0.   0.  50.   0.]
 [  0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]]


In [2]:
# inserting in locations some value using an index 
import theano 
import theano.tensor as T
import numpy
T_location = T.imatrix("location")
T_values = T.vector("values")
output_model = T.matrix("output_model")
T_i = T.ivector("i")

def step(i, output_model, a_location, a_values):
    return T.set_subtensor(output_model[a_location[i, 0], a_location[i, 1]], a_values[i])

result, updates = theano.scan(fn=step,
                              outputs_info=T.zeros((5, 5)),
                              sequences=[T_i],
                              non_sequences=[T_location, T_values])

assign_values_at_positions = theano.function(inputs=[T_i, T_location, T_values], outputs=result[-1])

# test
test_locations = numpy.asarray([[1, 1], [2, 3]], dtype=numpy.int32)
test_values = numpy.asarray([42, 50], dtype=numpy.float32)
test_i = [0, 1]
print(assign_values_at_positions(test_i, test_locations, test_values))


[[  0.   0.   0.   0.   0.]
 [  0.  42.   0.   0.   0.]
 [  0.   0.   0.  50.   0.]
 [  0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]]


In [3]:
# Window function with some operator... no stide here
import theano 
import theano.tensor as T
import numpy

T_location = T.imatrix("location") # indices of window in image
T_images = T.fmatrix("images")
T_stride = T.scalar("stride")
T_fsize = T.iscalar("fsize")


def step(location, images, fsize):
    img = images[location[0]:location[0] + fsize, location[1]:location[1] + fsize]
    ope = T.sum(img) # add some operation 
    return ope

result, updates = theano.scan(fn=step,
                              sequences=[T_location],
                              non_sequences=[T_images, T_fsize])

f = theano.function(inputs=[T_location, T_images, T_fsize], outputs=result)

# test
test_locations = numpy.asarray([[1, 1], [2, 3]], dtype=numpy.int32)
test_images = numpy.random.random((10, 10))
test_images = test_images.astype(numpy.float32)
test_fsize = 3
print(f(test_locations, test_images, test_fsize))




[ 4.09767962  4.22223425]


In [4]:
# Window function2 - "Sum" - convolution
import theano 
import theano.tensor as T
import numpy

T_location = T.imatrix("location") # indices of window in image
T_images = T.fmatrix("images")
T_stride = T.scalar("stride")
T_fsize = T.iscalar("fsize")
T_i = T.iscalar("index")
T_j = T.iscalar("index2")


def step(i, j, images, fsize, stride):
    img = images[i:i + fsize, j:j + fsize]
    ope = T.sum(img) # add some operation 
    s = images.shape[1] - fsize + 1
    
    i = i + (j+stride) // s
    j = (j+stride) % s
    j = j.astype("int32"); i = i.astype("int32")
    return i, j, ope, img

result, updates = theano.scan(fn=step,
                              outputs_info=[T_i, T_j, None, None],
                              non_sequences=[T_images, T_fsize, T_stride], 
                              n_steps=32)

assign_values_at_positions = theano.function(inputs=[T_i, T_j, T_images, T_fsize, T_stride], outputs=result)

# test
test_images = numpy.zeros((10, 10))
test_images[0, 0] = 1
test_images[0, 1] = 1
test_images[0, 2] = 1
test_images[9, 9] = 1

test_images = test_images.astype(numpy.float32)

test_fsize = 3
i, j, x, img = (assign_values_at_positions(0, 0, test_images, test_fsize, 2))
print(i)
print(j)


[0 0 0 1 1 1 1 2 2 2 2 3 3 3 3 4 4 4 4 5 5 5 5 6 6 6 6 7 7 7 7 8]
[2 4 6 0 2 4 6 0 2 4 6 0 2 4 6 0 2 4 6 0 2 4 6 0 2 4 6 0 2 4 6 0]


In [5]:
# Window function3 - convolution - with stride and a weight matrix
# for some strides and image sizes this will not work....
# the patches have to fit perfectly in the image

import theano 
import theano.tensor as T
import numpy

T_images = T.fmatrix("images")
T_stride = T.scalar("stride")
T_fsize = T.iscalar("fsize")
T_i = T.iscalar("index")
T_j = T.iscalar("index2")
T_w = T.fmatrix("W")


def step(i, j, images, fsize, stride, W):
    img = images[i:i + fsize, j:j + fsize]
    
    ope = T.sum(T.dot(img, W)) # add some operation  
    s = images.shape[1] - fsize + 1
    
    i = i + (j+stride) // s # works only in some cases (if window fits perfectly in image)
    j = (j+stride) % s
    j = j.astype("int32"); i = i.astype("int32")
    return i, j, ope

result, updates = theano.scan(fn=step,
                              outputs_info=[T_i, T_j, None],
                              non_sequences=[T_images, T_fsize, T_stride, T_w], 
                              n_steps=64)

assign_values_at_positions = theano.function(inputs=[T_i, T_j, T_images, T_fsize, T_stride, T_w], outputs=result)

# test
test_images = numpy.zeros((10, 10))
test_images[0, 0] = 1
test_images[0, 1] = 1
test_images[0, 2] = 1
test_images[9, 9] = 1

test_images = test_images.astype(numpy.float32)
test_W = numpy.zeros((3, 3))
test_W[0, 0] = 0.5
test_W[2, 2] = 0.3

test_W = test_W.astype(numpy.float32)
test_fsize = 3
i, j, conv = (assign_values_at_positions(0, 0, test_images, test_fsize, 1, test_W))
print(conv)

[ 0.80000001  0.5         0.5         0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.30000001]


In [6]:
# Window function3 - convolution
# always works
# works on multiple images (batches) and with images with multiple channels
# the weight matrix may contain multiple filters

import theano 
import theano.tensor as T
import numpy
from theano.ifelse import ifelse

T_images = T.ftensor4("images") # NFHW
T_stride = T.iscalar("stride")
T_fsize = T.iscalar("fsize")
T_i = T.iscalar("index")
T_j = T.iscalar("index2")
T_w = T.ftensor4("W") #FhwC


def step(i, j, images, fsize, stride, W):
    img = images[:, :, i:i + fsize, j:j + fsize] # NFhw

    ope = T.tensordot(img, W, axes=([1, 2, 3], [0, 1, 2])) # add some operation  
    s = images.shape[2] - fsize + 1
    
    i = i + ((j+stride) // s)
    j = ifelse(T.ge(j+stride, s), j-j, j+stride)
    
    j = j.astype("int32"); i = i.astype("int32")
    return i, j, ope

result, updates = theano.scan(fn=step,
                              outputs_info=[T_i, T_j, None],
                              non_sequences=[T_images, T_fsize, T_stride, T_w], 
                              n_steps=64)

assign_values_at_positions = theano.function(inputs=[T_i, T_j, T_images, T_fsize, T_stride, T_w], outputs=result)

# test
test_images = numpy.zeros((2, 3, 11, 11))
test_images[0, 0, 0, 0] = 1
test_images[0, 0, 0, 1] = 1
test_images[0, 0, 0, 2] = 1
test_images[0, 0, 9, 9] = 1

test_images = test_images.astype(numpy.float32)
test_W = numpy.zeros((3, 3, 3, 4))
test_W[0, 0, 0] = 0.5
test_W[2, 2, 0] = 0.3

test_W = test_W.astype(numpy.float32)
test_fsize = 3
i, j, conv = (assign_values_at_positions(0, 0, test_images, test_fsize, 1, test_W))
print(i)
print(j)
print(conv.shape)

[0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 4 4
 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 7 7]
[1 2 3 4 5 6 7 8 0 1 2 3 4 5 6 7 8 0 1 2 3 4 5 6 7 8 0 1 2 3 4 5 6 7 8 0 1
 2 3 4 5 6 7 8 0 1 2 3 4 5 6 7 8 0 1 2 3 4 5 6 7 8 0 1]
(64, 2, 4)


In [7]:
# test theanos tensordot on a single convolution
T_images = T.ftensor4("images") # NFHW
T_w = T.ftensor4("W") #FhwC


ope = T.tensordot(T_images, T_w, axes=([1, 2, 3], [0, 1, 2])) # add some operation  

f = theano.function(inputs=[T_images, T_w], outputs=ope)

# test
test_images = numpy.zeros((2, 3, 3, 3))
test_images[0, 0, 0, 0] = 1
test_images[0, 0, 0, 1] = 1
test_images[0, 0, 0, 2] = 1
test_images = test_images.astype(numpy.float32)
test_W = numpy.zeros((3, 3, 3, 4)) 
test_W[0, 0, 0] = 0.5
test_W[2, 2, 0] = 0.3
test_W = test_W.astype(numpy.float32)

x = (f(test_images, test_W))
print(x.shape)

(2, 4)
